# **ETL for Credit Card Churn Analysis**

## Objectives

* The workbook aims to load, clean datasets related to Credit Card Churn Analysis project. This aims to make use of KaggleAPI, typical data cleansing methods expected to produce a ready to use clean data file(s).  

## Inputs

* kaggle based credit-card-customers dataset (sakshigoyal7/credit-card-customers)

## Outputs

* Cleansed file of current & churned credit card customers & there attributes. 

## Additional Comments

* This will only contain the ETL elements of this process. Please see the other noteboot for vis details 



---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

print("Current Working Directory:", current_dir)   
  


Current Working Directory: c:\Users\ryan_\VS-code-projects\CreditCardChurn\jupyter_notebooks


We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))

print("You set a new current directory")


You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\ryan_\\VS-code-projects\\CreditCardChurn'

In [4]:
dwpath = os.path.join(current_dir, 'dataFiles', 'rawdata')
outpath = os.path.join(current_dir,'dataFiles', 'cleandata')
print("Data Download Path:", dwpath)    
print("Data Cleansed Path:", outpath)

Data Download Path: c:\Users\ryan_\VS-code-projects\CreditCardChurn\dataFiles\rawdata
Data Cleansed Path: c:\Users\ryan_\VS-code-projects\CreditCardChurn\dataFiles\cleandata


# Section 1 - Pull the dataset to local system

This section aims to pull the dataset from Kaggle.com's website via there API. 
(an account was previously created on website and api json file was generated and saved on my system)

In [5]:

from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

api.dataset_download_files('sakshigoyal7/credit-card-customers', path = dwpath, unzip= True)



Dataset URL: https://www.kaggle.com/datasets/sakshigoyal7/credit-card-customers


In [7]:
#checking the file downnloaded is accessible and has records. 
import pandas as pd
import numpy as np
df_raw = pd.read_csv(os.path.join(dwpath,'BankChurners.csv'))

df_raw.head()

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

---

# Section 2

Section 2 content

todo
Inspect shape, column names, dtypes

Check missing values

Check duplicates

Confirm target variable (Attrition_Flag) distribution

justify reasons for steps taken at end. 

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---